# Convnets'e giriş

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28,28,1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# input olarak convnet shape tensörü alır (image_height, image_width, image_channels).
# bizim bu örnekte (28, 28, 1)

In [2]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,202 (407.04 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# eğitim
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 34ms/step - accuracy: 0.8814 - loss: 0.3743
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 38s 31ms/step - accuracy: 0.9856 - loss: 0.0470
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 43s 33ms/step - accuracy: 0.9901 - loss: 0.0324
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - accuracy: 0.9929 - loss: 0.0245
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - accuracy: 0.9949 - loss: 0.0168


In [6]:
# evaluating the convnet
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy : {test_acc:.3f}")
# ilk chapterlarda yaptığımızda %97.8 civarlarında bir başarı vardı fakat şu anda %99.2

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9881 - loss: 0.0330
Test accuracy : 0.992


# Konvolüsyon operasyonu
Önceki densely connected layer yönteminde piksel piksel bakılarak bir pattern aranıyordu. Fakat Konvolüsyon ile yerel pattern'ler bulunuyor.

2 önemli karakteristik özellik var:
1. "The patterns they learn are translation-invariant". Yani sağ altta bir pattern gördüyse bu pattern'i nerede görürse görsün tanır. Sol üstte olması, ortada olması önemli değildir. Önceki yöntemimizde bu böyle değildi, aynı pattern farklı bir yerdeyse bunu tekrardan öğrenmesi gerekiyordu. Bu büyük bir özellik çünkü artık görsel verilerimizin tamamı yerden/konumdan bağımsız.

2. "They can learn spatial hiearchies of patterns". İlk konvolüsyon katmanı koşeler gibi yerel pattern'leri öğreniyor, ikinci konvolüsyon katmanı daha büyük olan birinci katmanlarla oluşturulan daha büyük bir pattern'i öğreniyor. Böylelikle gittikçe öğrenilen pattern hiyerarşik olarak büyüyor. Görsel dünyadaki örüntüler hiyerarşik yapıdadır.



Konvolüsyon operasyonu rank-3 bir tensör üzerinden gerçekleştirilir. `Feature maps`, iki uzamsal özellik: `height (yükseklik)`, `width (genişlik)` ve derinlik ekseni `channel axis`. RGB (Red-Green-Blue) bu derinlik ekseni 3'tür, çünkü görüntüde üç farklı renk vardır. Siyah-beyaz görsellerde bu değer 1'dir.

Konvolüsyon işlemi, girişten küçük yama (patch) parçaları alır ve her yamaya aynı dönüşümü (filtreyi) uygular. Sonuç olarak `output feature map` oluşur. Bu çıktı yine üç boyutludur (yükseklik, genişlik, derinlik). Ancak artık bu çıktının derinliği RGB renkleri ile değil, filtreleri temsil eder.

Her bir filtre, giriş görüntüdeki belli bir deseni yakalamaya çalışır.
    - Bir filtre kenarları
    - Başkası köşeleri
    - Başkası yüz gibi daha karmaşık örüntüleri...


## MNIST örneğinde
- Giriş tensörü (28,28,1)
- ilk konvolüsyon katmanı, 32 filtre uygular
- her filtre, girişin üzerine kayarak (26,26) boyutunda bir çıktı üretir
- Böylece çıktı tensörü (26, 26, 32) olur.


## Feature map
Derinlik eksenindeki her bir boyut, bir özellik (feature ya da filtre) temsil eder.
Yani `output[:, :, n] şeklinde aldığın her 2D kısım, o filtrenin giriş üzerinde verdiği yanıtı gösterir.

`depth` boyutundaki her bir kanal = bir filtre = bir özellik haritası

## Konvolüsyonun 2 temel parametresi
1. Patch boyutu
    - Girişten alınan pencerenin boyutu (3x3 ya da 5x5)
2. Çıktı derinliği (output depth)
    - kaç farklı filtre uygulanacağını belirler (başlangıçta 32, sonra 64 filtre olabilir)

`Conv2D(output_depth, (window_height, window_width))`

##Konvolüsyon Nasıl Çalışır?

- (3, 3) veya (5, 5) boyutunda pencere (window) alır.

- Bu pencere, giriş tensörünün üzerinde kayarak her lokasyonda durur.

- Her lokasyonda bir 3D yama çıkarılır:

- `(window_height, window_width, input_depth)` şeklinde

- Bu yama, öğrenilmiş bir ağırlık matrisi (kernel) ile çarpılır.

- Bu kernel, bütün yamalarda aynı kalır.

- Çıktı: `output_depth` boyutlu bir vektör

- Tüm bu çıktılar, 3D bir output feature map'e birleştirilir:

- `(height, width, output_depth)`

## Çıktı Boyutu Neden Girişten Farklı Olabilir?

1. Sınır Etkileri (Border Effects):

  Görüntünün kenarında pencere tam oturamayabilir.

  Çözüm: padding="same" gibi yastıklama yapılır.

2. Stride (Adım Boyu):

  Pencerenin kaç pikselde bir kayacağını belirler.

  Örn: stride=2 → pencere her seferinde 2 piksel kayar → çıktı küçülür.


## Border effects ve padding'i anlayalım
Padding Neden Gerekli?

Eğer çıktının da girişle aynı boyutta olmasını istiyorsan, bu kaybı önlemelisin.
Bunun için:

- Girişin etrafına sıfırlarla dolu satır/sütunlar eklersin (padding).

- Böylece filtre her yerde rahatça dolaşabilir.

`Conv2D(filters=32, kernel_size=(3, 3), padding="same")`

- "valid" → padding yok → çıktı daha küçük

- "same" → gerekli padding eklenir → çıktı girişle aynı boyutta

$$
\text{Output size} = \frac{\text{Input size} - \text{Filter size}}{\text{Stride}} + 1
$$

# Konvolüsyon adımlarını (strides) anlayalım
Strides dediğimiz kavram filtrenin kaç piksel ilerleyeceğini belirtir. Varsayılan olarak 1'dir ve birer piksel olarak fırçamızı ilerletir.

Filtre daha az konumda uygulandığı için çıkış boyutu düşer.

Sınıflandırma modellerinde, feature maps'i azaltmak için strides yerine daha çok `max-pooling` kullanırız.

## Max-pooling operasyonu

- Max pooling, giriş olarak bir özellik haritası (feature map) alır. Bu, genellikle konvolüsyon katmanından çıkan veridir.

- Max pooling işlemi, bu özellik haritasını küçük pencerelere (window) böler.
- Genellikle bu pencere boyutu 2×2’dir.

- Her pencerede, penceredeki en büyük (maksimum) değer seçilir.

- Bu seçilen maksimum değer, o pencere için çıktı olur.

In [7]:
# max-pooling katmanı olmadan hatalı bir şekilde convnet oluşturmak

inputs = keras.Input(shape=(28,28,1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

In [8]:
model_no_max_pool.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 22, 22, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 61952)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │       619,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 712,202 (2.72 MB)

 Trainable params: 712,202 (2.72 MB)

 Non-trainable params: 0 (0.00 B)

Burada iki problem var:
1. **Model çok küçük alanlara bakıyor:**  
Son katmandaki filtreler girişin sadece küçük bir parçasını (7×7 piksel) görebiliyor. Bu yüzden daha büyük, karmaşık desenleri tam öğrenemiyor.

2. **Parametre sayısı çok fazla:**  
Son konvolüsyon katmanından çıkan 22×22×128 boyutundaki veri flatten edilince, Dense katmanda yarım milyondan fazla parametre oluyor. Bu da aşırı öğrenmeye yol açıyor.


**Eğer max-pooling olsaydı:**

- Max-pooling feature map’in boyutunu örneğin 2 kat küçültür (22×22 → 11×11 gibi).
- Böylece son katmandaki çıktı daha küçük olur, parametre sayısı önemli ölçüde azalır.
- Ayrıca model, farklı seviyelerdeki özellikleri daha geniş alanlarda görebilir; yani mekânsal hiyerarşi öğrenmesi kolaylaşır.
- Sonuç olarak model daha verimli öğrenir, aşırı öğrenme riski düşer.
